In [1]:
import torch

In [2]:
import numpy as np

In [3]:
from torchtext import data, datasets

In [12]:
import random

In [4]:
seed = 966

In [5]:
torch.manual_seed(seed)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cpu')

In [10]:
TEXT = data.Field(tokenize="spacy", tokenizer_language="en_core_web_sm",lower=True)
LABEL = data.LabelField()

In [13]:
train, test = datasets.TREC.splits(TEXT, LABEL)
train, val = train.split(random_state=random.seed(seed))

In [14]:
vars(train[-1])

{'text': ['how', 'do', 'you', 'say', '2', 'in', 'latin', '?'], 'label': 'ENTY'}

In [15]:
TEXT.build_vocab(train, min_freq=2)

In [17]:
LABEL.build_vocab(train)

In [18]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, val, test), 
    batch_size = 64,
    sort_key = lambda x: len(x.text),
    device=device
)

In [19]:
import torch.nn as nn
import torch.nn.functional as F


In [24]:
class CNN(nn.Module):
    
    def __init__(
        self, vocabulary_size, embedding_size, kernels_number, kernel_sizes,
        output_size, dropout_rate):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.convolution_layers = nn.ModuleList(
            nn.ModuleList(
                [
                    nn.Conv2d(
                    in_channels=1, 
                    out_channels=kernels_number, 
                    kernel_size=(k, embedding_size))
                for k in kernel_sizes
            ]))
        self.dropout = nn.Dropout(dropout_rate)
        self.fully_connected = nn.Linear(
            len(kernel_sizes) * kernels_number, output_size)
    
    def forward(self, text):
        
        text = text.permut(1,0)
        input_embeddings = self.embedding(text)
        input_embeddings = input_embeddings.unsqueeze(1)
        conved = [
            F.relu(convolution_layer(input_embeddings)).squeeze(3)
            for convolution_layer in self.convolution_layers
        ]
        
        pooled = [
            F.max_pool1d(conv, conv.shape[2]).squeeze(3) for conv in conved
        ]
        
        concat = self.dropout(torch.cat(pooled, dim=1))
        
        final_output = self.fully_connected(concat)
        
        return final_output

In [25]:
vocabulary_size = 2679
embedding_size = 100
kernels_number = 100
kernel_sizes = [2, 3, 4]
output_size = 6
dropout_rate = 0.3


In [26]:
model = CNN(
    vocabulary_size, embedding_size, kernels_number, kernel_sizes, 
    output_size, dropout_rate)

In [27]:
print(model)

CNN(
  (embedding): Embedding(2679, 100)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (fully_connected): Linear(in_features=300, out_features=6, bias=True)
)


In [28]:
model.to(device)

CNN(
  (embedding): Embedding(2679, 100)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (fully_connected): Linear(in_features=300, out_features=6, bias=True)
)